In [1]:
from __future__ import division 
import matplotlib.pyplot as plt
%matplotlib inline
import emotion_analysis
emo = emotion_analysis.EmotionAnalysis()

In [2]:
inp, lab = emo.prepare_dataset_per_frame('data')

Read audio file 'data\positive_252.wav', count: 100


In [3]:
from collections import Counter
Counter(lab)

Counter({'negative': 62847, 'neutral': 46951, 'positive': 80586})

In [4]:
#from random import shuffle
import numpy as np
#dataset = zip(inp, lab)
#shuffle(dataset)
#inp, lab = zip(*dataset)
max_value = max([max(i) for i in inp])
min_value = min([min(i) for i in inp])

#inp = inp[:100000]
#lab = lab[:100000]

#normalised_inp = np.asarray(inp)/max_value
inp = (np.asarray(inp)-min_value)/(max_value-min_value)

In [5]:
# Random Forest : Better performance

#from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(n_estimators=100)
#clf = clf.fit(inp[0:int(len(inp)*3/float(4))],lab[0:int(len(inp)*3/float(4))])
#clf = clf.fit(inp, lab)

In [6]:
# SVM with RBF : Worst performance

from sklearn import svm
clf = svm.SVC(kernel = 'rbf')
clf = clf.fit(inp[0:int(len(inp)*3/float(4))],lab[0:int(len(inp)*3/float(4))])
#clf = clf.fit(inp, lab)

In [ ]:
y_pred_train = [clf.predict([point]) for point in inp[0:int(len(inp)*3/float(4))]]
y_actual_train = [output for output in lab[0:int(len(inp)*3/float(4))]]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(y_actual_train, y_pred_train, labels=['neutral','positive','negative'])
print "Precision = "+str(precision)+"\nRecall = "+str(recall)+"\nF1 Score = "+str(f1_score)

In [ ]:
y_pred_test = [clf.predict([point]) for point in inp[int(len(inp)*3/float(4)):]]
y_actual_test =[output for output in lab[int(len(inp)*3/float(4)):]]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(y_actual_test, y_pred_test, labels=['neutral','positive','negative'])
print "Precision = "+str(precision)+"\nRecall = "+str(recall)+"\nF1 Score = "+str(f1_score)

In [ ]:
lab_to_int = {'neutral':0, 'positive':1, 'negative':2}
int_to_lab = {0:'neutral', 1:'positive', 2:'negative'}

In [ ]:
import feature_extractor
import scipy.io.wavfile as wav
import math

extractor = feature_extractor.FeatureExtractor()

def final_result(audio_signal):
    all_result = []
    for frame_feature in extractor.extract_features_per_frame(audio_signal = audio_signal):
        frame_feature = (np.asarray(frame_feature)-min_value)/(max_value-min_value)
        frame_result = clf.predict([frame_feature])
        all_result.append(frame_result[0])
    label_counts = Counter(all_result)
    return max(label_counts, key = label_counts.get)

In [ ]:
from os import listdir
from os.path import isfile, join
import scipy.io.wavfile as wav
from random import shuffle

path = 'data'

y_pred=[]
y_actual=[]

audio_files = [f for f in listdir(path) if isfile(join(path, f))]
shuffle(audio_files)
count = 0
for audio_file in audio_files:
    count+=1
    print "Counter: "+str(count)
    y_pred.append(final_result(audio_signal = wav.read(join(path, audio_file))))
    y_actual.append(audio_file.split('_')[0])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(y_actual, y_pred, labels=['neutral','positive','negative'])
print "Precision = "+str(precision)+"\nRecall = "+str(recall)+"\nF1 Score = "+str(f1_score)

In [ ]:
file_name = raw_input("Enter the name of the call file name.\n> ")
rate_sig, sig = wav.read(file_name)

plt.plot(sig)
plt.plot((0,len(sig)),(0,0),'g')
plt.show()

complete_call_result = final_result(audio_signal = (rate_sig, sig))
first_half_result = final_result(audio_signal = (rate_sig, sig[:int(len(sig)/2)]))
second_half_result = final_result(audio_signal = (rate_sig, sig[int(len(sig)/2):]))
segment_result = []
for i in range(int(math.ceil(len(sig)/(rate_sig*20)))):
    segment_result.append(final_result(audio_signal = (rate_sig, sig[i*rate_sig*20:i*rate_sig*20+rate_sig*20])))

In [ ]:
# import numpy as np
from __future__ import division
print "Overall call result: "+complete_call_result
print "Trainsition: "+first_half_result+" --> "+second_half_result
map_dict = {'negative':-1, 'neutral':0, 'positive':1}
mapped_segment_result = [map_dict[result] for result in segment_result]
index = [i for i in range(len(mapped_segment_result))]
index = np.asarray(index)
bar_width = 1
plt.plot((0,index[-1]+1),((0,0)), 'r')
plt.bar(index, mapped_segment_result, bar_width)
plt.xticks(index + bar_width/2 , (str(i*20)+'-'+str((i+1)*20) for i in index))
plt.xlabel('Time series with 20 seconds chunk')
#plt.axes([0, 100, -2, 2])
plt.ylabel('Emotion:\n1 = positive\n0 = neutral\n-1 = negative')
plt.title('Emotion per 20 second segment')
plt.show()

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(cm))
    plt.xticks(tick_marks, ['neutral','positive','negative'], rotation=45)
    plt.yticks(tick_marks, ['neutral','positive','negative'])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_actual, y_pred, labels=['neutral','positive','negative'])
print cm
plot_confusion_matrix(cm)

In [ ]:
len([1 for i in range(len(y_pred)) if y_pred[i]==y_actual[i]])/float(len(y_pred))

In [ ]:
lab[1] 